In [6]:
import pandas as pd
import numpy as np
import os
import re

#set data paths
os.chdir("/Users/quinnmackay/Documents/GitHub/BICC/Holcene Revision Work")
EDML_age = 'EDML LC/Data Files/EDML Layer Count.xlsx'
GICC_GRIP_age = "EDML LC/Data Files/GICC05-GICC21 Conversion.xlsx"
links_path = 'EDML LC/Data Files/NGRIP_EDML_match.xlsx'

#import data
EDML = pd.read_excel(EDML_age, sheet_name=1, skiprows=1, names=["Depth (m)", "count", 'Year b2k', 'MCE'], usecols=[0,1,2,3])
EDML["age (b1950)"] = EDML["Year b2k"] - 50

#get count vals
EDML['total 0.5'] = (EDML['count'] == 0.5).cumsum()

EDML = EDML[EDML['age (b1950)'] <= 3619.36]

In [7]:

layer_file = pd.read_excel('EDML LC/EDML-GICC Errors/New_Layers.xlsx', usecols=[8,9,10,11])

current_output = 0
new_layers = []
remove_layers = []
half_add = []
half_remove = []

for i in range(17):  # cycle through rows

    # --- New_layer(m) ---
    val = layer_file["New_layer(m)"][i]
    if pd.isna(val):
        pass
    elif isinstance(val, str):  # string, split into multiple values
        tempvals = [v.strip() for v in val.split(",")]
        for v in tempvals:
            new_layers.append(float(v))
            current_output += 1
    else:  # already numeric
        new_layers.append(float(val))
        current_output += 1

    # --- Remove_layer(yr) ---
    val = layer_file["Remove_layer(yr)"][i]
    if pd.isna(val):
        pass
    elif isinstance(val, str):
        tempvals = [v.strip() for v in val.split(",")]
        for v in tempvals:
            remove_layers.append(float(v))
            current_output -= 1
    else:
        remove_layers.append(float(val))
        current_output -= 1

    # --- Half_add ---
    val = layer_file["Half_add"][i]
    if pd.isna(val):
        pass
    elif isinstance(val, str):
        tempvals = [v.strip() for v in val.split(",")]
        for v in tempvals:
            half_add.append(float(v))
            current_output += 0.5
    else:
        half_add.append(float(val))
        current_output += 0.5

    # --- Half_remove ---
    val = layer_file["Half_remove"][i]
    if pd.isna(val):
        pass
    elif isinstance(val, str):
        tempvals = [v.strip() for v in val.split(",")]
        for v in tempvals:
            half_remove.append(float(v))
            current_output -= 0.5
    else:
        half_remove.append(float(val))
        current_output -= 0.5

    print(f"Row {i} -> current_output: {current_output}")


Row 0 -> current_output: -1.5
Row 1 -> current_output: -1.0
Row 2 -> current_output: -1.5
Row 3 -> current_output: 8.5
Row 4 -> current_output: 9.5
Row 5 -> current_output: 8.5
Row 6 -> current_output: 12.5
Row 7 -> current_output: 12.0
Row 8 -> current_output: 13.5
Row 9 -> current_output: 16.5
Row 10 -> current_output: 7.5
Row 11 -> current_output: 8.0
Row 12 -> current_output: 5.5
Row 13 -> current_output: -8.5
Row 14 -> current_output: -7.0
Row 15 -> current_output: -16.5
Row 16 -> current_output: -17.5


In [8]:
#now let's write confirmation code to make sure all the 0.5s are correct

add_row_idx = []
for val in half_add:
    if val in EDML['age (b1950)'].values:
        matches = EDML.index[EDML['age (b1950)'] == val].tolist()
        add_row_idx.extend(matches)
    elif val not in EDML['age (b1950)'].values:
        print(f"Error: Value {val} from half_add not found in EDML ages.")

remove_row_idx = []
for val in half_remove:
    if val in EDML['age (b1950)'].values:
        matches = EDML.index[EDML['age (b1950)'] == val].tolist()
        remove_row_idx.extend(matches)
    elif val not in EDML['age (b1950)'].values:
        print(f"Error: Value {val} from half_remove not found in EDML ages.")

for idx in add_row_idx:
    if EDML.loc[idx, 'count'] == 0.5:
        continue
    elif EDML.loc[idx, 'count'] == 1:
        print(f"Error: Count is 1 at index {idx} for half_add value {EDML.loc[idx, 'age (b1950)']}")
    else:
        print(f"Something else major is wrong at index {idx} for half_add value {EDML.loc[idx, 'age (b1950)']}")

for idx in remove_row_idx:
    if EDML.loc[idx, 'count'] == 0.5:
        continue
    elif EDML.loc[idx, 'count'] == 1:
        print(f"Error: Count is 1 at index {idx} for half_remove value {EDML.loc[idx, 'age (b1950)']}")
    else:
        print(f"Something else major is wrong at index {idx} for half_remove value {EDML.loc[idx, 'age (b1950)']}")


all_half_vals = EDML[EDML['count'] == 0.5]['age (b1950)'].tolist()
for half in all_half_vals:
    if half not in half_add and half not in half_remove:
        print(f"Error: Half value {half} in EDML not accounted for in half_add or half_remove.")

# for the 1s
remove_row_idx_full = []
for val in remove_layers:
    if val in EDML['age (b1950)'].values:
        matches = EDML.index[EDML['age (b1950)'] == val].tolist()
        remove_row_idx_full.extend(matches)
    elif val not in EDML['age (b1950)'].values:
        print(f"Error: Value {val} from remove_layers not found in EDML ages.")

In [9]:
New_EDML = EDML[["Depth (m)", "count", "age (b1950)", "MCE"]].copy(deep=True)

# apply the add/remove changes, dropping remove rows and setting add rows to 1
New_EDML.loc[add_row_idx, 'count'] = 1
New_EDML = New_EDML.drop(index=remove_row_idx)

# add new layers
for val in new_layers:
    if val in New_EDML['Depth (m)'].values:
        print(f"Error: New layer depth {val} already exists in New_EDML.")
    else:
        new_row = pd.DataFrame({'Depth (m)': [val], 'count': [1]})
        New_EDML = pd.concat([New_EDML, new_row], ignore_index=True)

#remove layers
for val in remove_layers:
    mask = np.isclose(New_EDML['age (b1950)'], val, atol=1e-3)
    if mask.any():
        idx_to_remove = New_EDML.index[mask].tolist()
        New_EDML = New_EDML.drop(index=idx_to_remove)
    else:
        print(f"Error: Remove layer age {val} not found in New_EDML.")

#sort by depth to get in order
New_EDML = New_EDML.sort_values(by="Depth (m)", ascending=True, ignore_index=True)

#set NaN MCE values to equal previous value
New_EDML['MCE'] = New_EDML['MCE'].ffill()

#reset depths
EDML_reset_depth = New_EDML.copy(deep=True)
EDML_reset_depth['age (b1950)'] = np.nan  # assuming 0.02 m intervals
EDML_reset_depth.at[EDML_reset_depth.index[0], 'age (b1950)'] = New_EDML.at[New_EDML.index[0], 'age (b1950)']

#age adjustments
for i in range(1, len(EDML_reset_depth)):
    EDML_reset_depth.at[EDML_reset_depth.index[i], 'age (b1950)'] = EDML_reset_depth.at[EDML_reset_depth.index[i-1], 'age (b1950)'] + 1

#full age adjustment - adding 0.5 to all, to make it in even intervals. Can adjust later if needed.
EDML_reset_depth['age (b1950)'] = EDML_reset_depth['age (b1950)'] + 0.5

EDML_reset_depth.to_csv('EDML LC/EDML-GICC Errors/EDML_LayerCount_Revised.txt', index=False, sep='\t')

In [10]:
# recalculate EDML-GICC Errors - sourced from the EDML-GICC Errors script



links = pd.read_excel(links_path, skiprows=11, names=["GICC05 Age", "EDML (m)", "NGRIP (m)", "Max Link Error"], usecols=[0,1,2,3])

#links data GICC21
GICC21_transfer = pd.read_excel(GICC_GRIP_age, sheet_name=4, skiprows=8, usecols=[0,2], names=["GICC21 age", "GICC05 age"])
links['GICC21 Age b1950'] = (np.interp(links["GICC05 Age"], GICC21_transfer['GICC05 age'], GICC21_transfer['GICC21 age']))-50


EDML_GICC_compare = links[["EDML (m)"]].copy() #create a new dataframe for WDC and GRIP depths using stratigraphic links

#get interp age for depths from layer count
interp_edml_age = np.interp(EDML_GICC_compare["EDML (m)"], EDML_reset_depth["Depth (m)"], EDML_reset_depth["age (b1950)"]) #interpolate depths to get ages

#add interpolated ages to dataframe
EDML_GICC_compare["EDML age (yr BP1950)"] = interp_edml_age #add interpolated ages to dataframe
EDML_GICC_compare = EDML_GICC_compare.reset_index(drop=True) # reset the index

# add NGRIP depths and ages to dataframe
EDML_GICC_compare["NGRIP (m)"] = links["NGRIP (m)"].copy(deep=True) #add NGRIP depths to dataframe
EDML_GICC_compare["NGRIP age (yr BP1950)"] = links["GICC21 Age b1950"].copy(deep=True) #add NGRIP ages to dataframe

# limit to LC and GICC21 range
EDML_GICC_compare = EDML_GICC_compare[EDML_GICC_compare["NGRIP age (yr BP1950)"] > 1250] #limit to LC and GICC21 range
EDML_GICC_compare = EDML_GICC_compare[EDML_GICC_compare["NGRIP age (yr BP1950)"] < 3610] 

#calc diff
EDML_GICC_compare["difference (yr)"] = EDML_GICC_compare["EDML age (yr BP1950)"] - EDML_GICC_compare["NGRIP age (yr BP1950)"] #calculate difference between ages

# this computes the error from that given section, which is the difference between layer counts in GICC and WDC for that section
section_error = np.zeros(len(EDML_GICC_compare)) #create empty list for section error
for i in range(0, len(EDML_GICC_compare)):
    if i == 0:
        EDML_error = EDML_GICC_compare["EDML age (yr BP1950)"].iloc[i] - 0 #calculate section error for each row
        GICC_error = EDML_GICC_compare["NGRIP age (yr BP1950)"].iloc[i] - 0 #calculate section error for each row
    else:
        EDML_error = EDML_GICC_compare["EDML age (yr BP1950)"].iloc[i] - EDML_GICC_compare["EDML age (yr BP1950)"].iloc[i-1] #calculate section error for each row
        GICC_error = EDML_GICC_compare["NGRIP age (yr BP1950)"].iloc[i] - EDML_GICC_compare["NGRIP age (yr BP1950)"].iloc[i-1] #calculate section error for each row
    section_error[i] = EDML_error - GICC_error #calculate section error for each row

EDML_GICC_compare["section error (EDML-NGRIP) yr"] = section_error #calculate difference between ages

output_path = "EDML LC/EDML-GICC Errors/NEW_LC_EDML_GICC_Compare.xlsx"
EDML_GICC_compare.to_excel(output_path, index=False)

print(f"File saved to: {output_path}")

File saved to: EDML LC/EDML-GICC Errors/NEW_LC_EDML_GICC_Compare.xlsx
